In [ ]:

import os
os.environ["WANDB_DISABLED"] = "true"

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [3]:
!pip install transformers datasets peft trl

In [4]:
 from transformers import (
     AutoModelForCausalLM,
     AutoTokenizer,
     BitsAndBytesConfig,
     TrainingArguments,
   
 )
 import torch
 from datasets import load_dataset
 from peft import LoraConfig, get_peft_model
 from trl import SFTTrainer

2025-08-17 11:28:39.236563: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755430119.258732     161 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755430119.265623     161 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [5]:
# pip install -U transformers


In [ ]:
from huggingface_hub import login

# paste your HF token here
login("YOUR_KEY")


In [7]:
 model_name = "meta-llama/Llama-3.2-1B-Instruct"

 # Tokenizer setup
 tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
 tokenizer.pad_token = tokenizer.eos_token
 tokenizer.padding_side = "right"


model = AutoModelForCausalLM.from_pretrained(model_name,
     device_map="auto"
 )


In [ ]:
 from peft import LoraConfig, get_peft_model

 lora_config = LoraConfig(
     lora_alpha=1024,
     lora_dropout=0.5,
     r=1024,  #adjust rank accoring to your need
     bias="none",
     task_type="CAUSAL_LM",
 )

 model = get_peft_model(model, lora_config)

In [10]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("Hemanth-thunder/english-to-odia-mt")


In [11]:
ds

DatasetDict({
    train: Dataset({
        features: ['en', 'or'],
        num_rows: 69370
    })
})

In [12]:

# Assuming your dataset is called `ds`
columns_to_keep = [ "en", "or"]
# columns_to_remove = [c for c in ds_train.column_names if c not in columns_to_keep]

colums_to_remove=[c for c in ds['train'].column_names if c not in columns_to_keep]

ds_train = ds['train'].remove_columns(colums_to_remove)

print(ds_train)

Dataset({
    features: ['en', 'or'],
    num_rows: 69370
})


In [13]:
# new_ds=[]
# for i in ds_train:
#     print(i)
#     print['']
#     break

In [14]:
ds_train[0]

{'en': 'That they may keep thee from the strange woman, from the stranger which flattereth with her words.',
 'or': 'ତବେେ ତାହା ତୁମ୍ଭକୁ ଅନ୍ୟ ପର ସ୍ତ୍ରୀଠାରୁ ରକ୍ଷା କରିବ। ଏବଂ ବ୍ଯଭିଚାରିଣୀ ସ୍ତ୍ରୀଙ୍କଠାରୁ ମଧ୍ଯ ରକ୍ଷା କରିବ।'}

In [15]:
def preprocess_function(example):

    return {
        "prompt": [{"role": "user", "content": example["en"]}],
        "completion": [
            {"role": "assistant", "content": example['or']}
        ],
    }

dataset = ds_train.map(preprocess_function,remove_columns= ['en','or'])
print(dataset[0])






{'prompt': [{'content': 'That they may keep thee from the strange woman, from the stranger which flattereth with her words.', 'role': 'user'}], 'completion': [{'content': 'ତବେେ ତାହା ତୁମ୍ଭକୁ ଅନ୍ୟ ପର ସ୍ତ୍ରୀଠାରୁ ରକ୍ଷା କରିବ। ଏବଂ ବ୍ଯଭିଚାରିଣୀ ସ୍ତ୍ରୀଙ୍କଠାରୁ ମଧ୍ଯ ରକ୍ଷା କରିବ।', 'role': 'assistant'}]}


In [16]:
dataset[4]['completion'][0]['content']


'ଆଜି ରାତ୍ରିର କଥା କିନ୍ତୁ ସ୍ଵତନ୍ତ୍ର ।'

In [17]:
tokenizer.decode(tokenizer.encode(dataset[4]['completion'][0]['content']))

'<|begin_of_text|>ଆଜି ରାତ୍ରିର କଥା କିନ୍ତୁ ସ୍ଵତନ୍ତ୍ର ।'

In [18]:
 from transformers import Trainer, TrainingArguments
 from trl import SFTTrainer

 training_args = TrainingArguments(
     output_dir="./results",
     eval_strategy="no",
     learning_rate=2e-5,
     per_device_train_batch_size=2,
     num_train_epochs=1,
     # dataset_text_field="text",
     fp16=True,
     save_total_limit=2,
     logging_steps=1000,
     save_steps=2500,

 )

 trainer = SFTTrainer(
     model= model,
     train_dataset=dataset,
    #  peft_config= lora_config,
     # dataset_text_field="text",
     # max_seq_length=512,  # Set max sequence length
     # tokenizer=tokenizer,
     args=training_args,
     # packing=False,
 )

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [19]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [20]:
trainer.train()

Step,Training Loss
1000,0.644600
2000,0.531500
3000,0.494400
4000,0.471200
5000,0.464500
6000,0.449200
7000,0.434700
8000,0.428300
9000,0.428600
10000,0.419300


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using

TrainOutput(global_step=34685, training_loss=0.4051505801621682, metrics={'train_runtime': 13479.3378, 'train_samples_per_second': 5.146, 'train_steps_per_second': 2.573, 'total_flos': 1.916339537595433e+17, 'train_loss': 0.4051505801621682})

## INFERENCE


In [ ]:
from transformers import pipeline
pipe = pipeline("text-generation", model=model,tokenizer=tokenizer)
prompt = [{'content': "what are you doing ?",'role': 'user'}]
response = pipe(prompt)
response[0]["generated_text"]